In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List
import networkx as nx
import random
from gensim.models import Word2Vec
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score
import numpy as np

from load_data import load_data, preprocess_features, run_regression
from deepwalk_node2vec import get_walk_paths, build_model,
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
dataset_dir = "../data/reddit"
train_ids, test_ids, val_ids, train_labels, test_labels, val_labels, gf = load_data(
    dataset_dir
)


Loading data...


In [4]:
train_feats, test_feats, val_feats = preprocess_features(dataset_dir, train_ids, test_ids, val_ids)

Using only features..


In [11]:
train_gf = gf.subgraph(train_ids[:50])


In [7]:

walks = get_walk_paths(train_gf, walks_per_node=5, n_steps_per_walk=6)


In [10]:

embedder = build_model(walks, progress_per=100000, epochs=100, min_count=0, workers=14)


2021-10-26 22:22:36,763 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=2, alpha=0.025)', 'datetime': '2021-10-26T22:22:36.763399', 'gensim': '4.1.2', 'python': '3.9.7 (default, Sep 10 2021, 14:59:43) \n[GCC 11.2.0]', 'platform': 'Linux-5.13.0-19-generic-x86_64-with-glibc2.34', 'event': 'created'}
2021-10-26 22:22:36,764 : INFO : collecting all words and their counts
2021-10-26 22:22:36,764 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-26 22:22:36,921 : INFO : PROGRESS: at sentence #100000, processed 697420 words, keeping 130893 word types
2021-10-26 22:22:37,054 : INFO : PROGRESS: at sentence #200000, processed 1394780 words, keeping 143070 word types
2021-10-26 22:22:37,181 : INFO : PROGRESS: at sentence #300000, processed 2091600 words, keeping 147258 word types
2021-10-26 22:22:37,307 : INFO : PROGRESS: at sentence #400000, processed 2789200 words, keeping 149328 word types
2021-10-26 22:22:37,436 : INFO : PROGRESS: a

In [11]:

logreg = SGDClassifier(loss="log", n_jobs=10)
embeddings = [embedder.wv[str(node)] for node in train_gf.nodes.keys()]
logreg.fit(embeddings, train_labels)
print("Validation scores")
print(f1_score(train_labels, logreg.predict(embeddings), average="micro"))

Validation scores
0.3005642674365199


In [12]:
ogreg = SGDClassifier(loss="log", n_jobs=10)
embeddings = np.concatenate(([embedder.wv[str(node)] for node in train_gf.nodes.keys()], train_feats), axis=1)
logreg.fit(embeddings, train_labels)
print("Validation scores")
print(f1_score(train_labels, logreg.predict(embeddings), average="micro"))

Validation scores
0.6179122104848763


In [ ]:
#could be reimplemented using https://github.com/Andras7/word2vec-pytorch/blob/master/word2vec/trainer.py